In [2]:
import gym
import gym_maze  # noqa: F401
import numpy as np
import matplotlib.pyplot as plt

import pathlib
import random
from itertools import groupby

from src.decorators import get_from_cache_or_run
from lcs import Perception

from src.visualization import PLOT_DPI
from src.utils import build_plots_dir_path, build_cache_dir_path

root_dir = pathlib.Path().cwd().parent.parent.parent
cwd_dir_name = pathlib.Path().cwd().name

plot_dir = build_plots_dir_path(root_dir) / cwd_dir_name
cache_dir = build_cache_dir_path(root_dir) / cwd_dir_name

plt.ioff();  # turn off interactive plotting

# Road towards Anticipatory Learning Classifier Systems
The concept of LCS was introduced by John Holland in 1975 {cite}`holland1975adaptation` in order to model the idea of cognition based on adaptive mechanisms. From the early days they consist of a set of rules named _classifiers_  combined with mechanisms in charge of evolving them. Initially the goal was to handle problems related to online interaction with external environments, as described by Wilson in {cite}`wilson1986knowledge`.

To accomplish this, the emphasis was put on parallelism in the architecture and evolutionary mechanisms allowing the agent to adapt to potentially changing environment {cite}`goldberg1988genetic`. This approach was referenced as _"escaping brittleness"_ {cite}`holland1986escaping` due to the problems related to the lack of robustness of the current artificial intelligence systems.

`````{margin}
````{admonition} John Holland
```{image} ../../_static/John_Holland.jpg
```

**John Henry Holland** (February 2, 1929 – August 9, 2015) was an American scientist and Professor of psychology and Professor of electrical engineering and computer science at the University of Michigan, Ann Arbor.

He was a pioneer in what became known as genetic algorithms.
````
`````

The naming convention used to refer to the LCS algorithm also changed since its infancy. Holland initially called it as a classifier system, abbreviated either as (CS) or (CFS) {cite}`robertson1988tale`. From that time it was also referred to as adaptive agents {cite}`holland1996hidden`, cognitive systems {cite}`holland1978cognitive` and genetic-based machine learning {cite}`dam2007neural` {cite}`goldberg1989`. The current name of LCS was not adopted until the late 80s {cite}`riolo1988empirical` after extending the architecture with a credit assignment component {cite}`holland1986escaping` {cite}`holland1986mathematical`.

This section provides a synopsis of crucial LCS concepts alongside the most popular variants, and the contributions they made to the field.


## Selected topics of LCS
### Rules and Classifiers

In general form, LCS utilizes _rules_ as a fundamental block of modeling knowledge. It comprises a _condition_ (i.e. specified feature states), and an _action_ (also referred to as the class). They can be interpreted using the "IF condition THEN action" logical expression. When the condition part is expressed with either boolean or nominal representation the generalization property using the "don't care" symbol # is possible. Two input situations are considered equivalent with respect to a given classifier if the specified (non-#) values in the condition match the corresponding attributes of the two situations

In addition to condition and action, a rule typically has a number of algorithm related parameter values associated with it (like its performance or expected reward). The term _classifier_ is used to describe a rule along with it associated parameters.

It is important to realise that LCS comprise a population of single rules, that collaboratively seek to cover the problem space. The number of classifiers needed to solve the particular problem depends on a number of factors like problem complexity or rule representation used.

### Driving Mechanisms
There are two fundamental components behind every LCS algorithm - the discovery and learning (credit assignment) component. Both of them have generated respective fields of study, but it is in the context of LCS that we wish to understand their function and purpose.

**Discovery component**

The discovery component is responsible for exploring the search space of the target problem with the purpose of discovering new rules. A vast majority of LCS algorithms employ some form of Evolutionary Computation, most often Genetic Algorithm (GA), employing Neo-Darwinist theory of natural selection {cite}`bull2005foundations` {cite}`goldberg1989`. The evolution of rules is modeled after the evolution of organisms using following biological analogies:

1. a code is used to represent the genotype/genome (condition),
2. a solution (phenotype) representation is associated with that genome (action),
3. a phenotype selection process (survival of the fittest) - the fittest organism (rule) has the greatest chances of reproducing and passing parts of its genome to offsprings,
4. certain genetic genome operators are utilized in order to chase after fitter organisms (rules) {cite}`sigaud2007learning` {cite}`holmes1999learning`.

Two genetic operators are typically used to alternate genome (rule) - mutation and crossover (recombination). The first one randomly modify an element in a genotype of an individual (rule), while the latter one recombine parts of two promising genotypes (rules) creating a new one. The selection pressure driving better organisms (rules) to reproduce more frequently depends on the fitness function. The fitness function quantifies the optimality of given rule, allowing it to be ranked across all rules in entire population.

GA implementation varies in certain LCS, but the overall scheme involves evaluating all available rules, selecting most promising offsprings (according to fitness value), applying genetic operators, introducing new offspring back to population set and finally removing excess or under-performing individuals.


**Learning component**

As mentioned earlier each classifier is accompanied by certain parameter values. The iterative update of them drives the process of LCS reinforcement by distributing any incoming reward signal to the classifiers that are accountable for it. This process serves two purposes:

1. identification of classifiers responsible for obtaining large future rewards,
2. encourage the discoverability of new rules (by directly affecting the fitness value)

The learning strategy depends on the nature of the problem and is realized differently in LCS implementations. In all cases, however the process is performed through trial-and-error interactions with the environment, where occasional immediate reward is used to generate the policy (state-action mapping of agent-environment interactions) maximizing long-term reward {cite}`sutton2018reinforcement` {cite}`harmon96reinforcementlearning` {cite}`richter2003reinforcement`.


### Functional cycle
The agent interacts with the environment in consecutive trials. Each trial consists of sequential steps usually executed as follows:

1. Filter population $[P]$ and select classifiers where condition matches environmental perception forming a _match-set_ $[M]$.
2. Determine action that will be executed (depending on the strategy).
3. Narrow-down the match-set by selecting only classifiers advocating proposed action.
4. Execute the action in the environment obtaining a new state
5. Refine classifiers by executing discovery and learning components.

The action selection and classifiers evolution phases are implemented individually in different LCS, but the main objective of reaching ideal generalization level is crucial to all of them. The system should find a population that covers the search space as compactly as possible without being detrimental to the optimality of behavior.

### Pittsburgh vs Michigan
One of the most fundamental distinction in LSC research is the way of storing knowledge by using two different approaches - _Michigan-style_ or _Pittsburgh-style_. The first ones were proposed by Holland {cite}`holland1978cognitive` while the latter one by Kenneth DeJong and his student {cite}`smith1980learning` {cite}`smith1983flexible`.

The fundamental distinction is the structure of an individual. In Michigan systems each individual is a classifier, in Pittsburgh each individual is a set of classifiers - see Figure {numref}`{number} <michigan-pittsburgh-fig>`.

:::{figure-md} michigan-pittsburgh-fig
<img src="../../_static/michigan_pittsburgh.png">

Differences between representing knowledge in both Michigan-style and Pittsburgh-style LCS. Figure taken from {cite}`bacardit2006learning`.
:::

Therefore, the classifiers in Michigan-style LCS are being continuously evaluated and evolved, while in Pittsburgh-style LCS the same process is much more complicated because the whole population needs to be assessed. Thus, Michigan-style systems are typically applied in interactive, online learning problems, while Pittsburgh ones are rather suitable for offline learning {cite}`bacardit2006learning`. This work focuses on the first class of LCS.

## Representative Michigan-style LCS

### CS-1
Two years after presenting the theoretical model of CS Holland and Reitman proposed its implementation CS-1 {cite}`holland1978cognitive`. The system realized the Darwinian principle of the survival of the fittest and was generating adaptive behaviour by maximizing reinforcement using the bucket brigade algorithm {cite}`holland1989induction`. Some internal mechanisms like the usage of a message list or interactions between evolving both single classifier and entire population were considered difficult {cite}`wilson1989critical` which signifies the need for improvements.

### ZCS
`````{margin}
````{admonition} Steward Wilson
```{image} ../../_static/steward_wilson.gif
```

[link](https://ieeexplore.ieee.org/author/37550361200)
````
`````
The _Zeroth Level Classifier_ (ZCS) introduced by Wilson in 1994 {cite}`wilson1994zcs` encompasses all LCS components while simplifying the CS-1 increasing its understandability and performance. The major changes was the removing of the internal message list (determining the rule's format entirely by the system interface) alongside with rule-bidding credit assignment (replacing it with Q-learning {cite}`watkins1989learning`).

Moreover, the classifier fitness was based on the accumulated reward that the agent can get from firing the classifier which gave rise to the _"strength-based"_ family of LCS. As a result the discovery component eliminates classifiers providing less reward than others from the population.

This premature converge onto suboptimal rules before space can be properly explores and stable population formed led Wilson to consider other ways in which this might be achieved.

(section-topics-history-xcs)=
### XCS
[wiki](https://en.wikipedia.org/wiki/Learning_classifier_system)
XCS {cite}`wilson1995classifier` {cite}`wilson1998generalization` {cite}`wilson1999state`

XCS uses niche-GA [Booker, 1985] meaning that GA acts in the action space $[A]$, instead of globally.

## Anticipatory Learning Classifier Systems
### ACS2

In [19]:
import lcs.agents.acs2 as acs2
from lcs.metrics import population_metrics

from typing import Optional, Tuple

def maze_metrics(agent, env):
    metrics = {}
    metrics.update(population_metrics(agent.population, env))
    return metrics

acs2_base_params = {
    'classifier_length': 8,
    'number_of_possible_actions': 8,
    'biased_exploration': 0,
    'metrics_trial_frequency': 1,
    'user_metrics_collector_fcn': maze_metrics
}

def run_acs2_explore_exploit(env, explore_trials, exploit_trials, **config):
    cfg = acs2.Configuration(**config)

    # explore phase
    agent = acs2.ACS2(cfg)
    metrics_explore = agent.explore(env, explore_trials)

    # exploit phase
    agent_exploit = acs2.ACS2(cfg, copy(agent.population))
    metrics_exploit = agent_exploit.exploit(env, exploit_trials)

    return (agent, metrics_explore), (agent_exploit, metrics_exploit)

def find_best_classifier(population: acs2.ClassifiersList, situation: Perception) -> Optional[acs2.Classifier]:
    match_set = population.form_match_set(situation)
    anticipated_change_cls = [cl for cl in match_set if cl.does_anticipate_change()]

    if len(anticipated_change_cls) > 0:
        return max(anticipated_change_cls, key=lambda cl: cl.fitness)

    return None

def build_fitness_and_action_matrices(env, population) -> Tuple:
    original = env.env.maze.matrix

    fitness = original.copy()
    action = original.copy().astype(str)

    action_lookup = {
        0: u'↑', 1: u'↗', 2: u'→', 3: u'↘',
        4: u'↓', 5: u'↙', 6: u'←', 7: u'↖'
    }

    for index, x in np.ndenumerate(original):
        if x == 0:  # path
            perception = env.env.maze.perception(index)
            best_cl = find_best_classifier(population, perception)

            if best_cl:
                fitness[index] = best_cl.fitness
                action[index] = action_lookup[best_cl.action]
            else:
                fitness[index] = -1
                action[index] = '?'

        if x == 1:  # wall
            fitness[index] = 0
            action[index] = '\#'

        if x == 9:  # reward
            # add 500 to make it more distinguishable
            fitness[index] = fitness.max() + 500
            action[index] = 'R'

    return fitness, action

def plot_policy(env, fitness_matrix, action_matrix):
    fig, ax = plt.subplots(1, 1, figsize=(14, 8))

    max_x, max_y = env.env.maze.matrix.shape

    # Render maze as image
    plt.imshow(fitness_matrix, interpolation='nearest', cmap='Reds', aspect='auto', extent=[0, max_x, max_y, 0])

    # Add labels to each cell
    for (y, x), val in np.ndenumerate(action_matrix):
        plt.text(x + 0.4, y + 0.5, "${}$".format(val))

    ax.set_title("Policy in Maze5 environment", fontsize=24)
    ax.set_xlabel('x', fontsize=18)
    ax.set_ylabel('y', fontsize=18)
    ax.set_xlim(0, max_x)
    ax.set_ylim(max_y, 0)
    ax.set_xticks(range(0, max_x))
    ax.set_yticks(range(0, max_y))
    ax.grid(True)
    fig.savefig(f'{plot_dir}/acs2-maze5-policy.png', dpi=PLOT_DPI)

@get_from_cache_or_run(cache_path=f'{cache_dir}/acs2_maze5.dill')
def run_acs2_in_maze5():
    env = gym.make('Maze5-v0')
    explore_phase, exploit_phase = run_acs2_explore_exploit(env, explore_trials=5000, exploit_trials=100, **acs2_base_params)
    return env, explore_phase, exploit_phase

# Run computation
env_, explore_, exploit_ = run_acs2_in_maze5()

# Plot the policy
fitness_matrix, action_matrix = build_fitness_and_action_matrices(env_, explore_[0].population)
plot_policy(env_, fitness_matrix, action_matrix)

:::{figure-md} maze5-fig
:class: full-width
<img src="../../_static/plots/2_selected_topics/acs2-maze5-policy.png">

Policy of Maze5. Saturation of red color reflects the best classifier fitness value.
:::

### MACS

In [20]:
import lcs.agents.macs.macs as macs

maze228_env = gym.make('Maze228-v0')

def _calculate_rotating_maze_knowledge(agent: macs.MACS, env):
    transitions = env.env.transitions
    covered_transitions = 0

    for p0, a, p1 in transitions:
        p0p = Perception(list(map(str, p0)))
        p1p = Perception(list(map(str, p1)))
        anticipations = list(agent.get_anticipations(p0p, a))

        # accurate classifiers
        if len(anticipations) == 1 and anticipations[0] == p1p:
            covered_transitions += 1

    return covered_transitions / len(transitions)

def _macs_metrics(agent: macs.MACS, env):
    population = agent.population
    return {
        'pop': len(population),
        'situations': len(agent.desirability_values),
        '0_cls': len([cl for cl in population if cl.action == 0 and cl.is_accurate]),
        '1_cls': len([cl for cl in population if cl.action == 1 and cl.is_accurate]),
        '2_cls': len([cl for cl in population if cl.action == 2 and cl.is_accurate]),
        'knowledge': _calculate_rotating_maze_knowledge(agent, env)
    }

@get_from_cache_or_run(cache_path=f'{cache_dir}/macs_maze228.dill')
def run_macs_in_maze228(env):
    cfg = macs.Configuration(classifier_length=9,
                        number_of_possible_actions=3,
                        feature_possible_values=[{'0', '1', '9'}] * 8 + [{'0', '9'}],
                        estimate_expected_improvements=True,
                        metrics_trial_frequency=10,
                        user_metrics_collector_fcn=_macs_metrics)

    agent = macs.MACS(cfg)
    metrics = agent.explore(env, 100)

    return agent, metrics

# run computations
macs_agent, macs_metrics = run_macs_in_maze228(maze228_env)

# visualization
random.seed(129)  # found experimentally
maze228_env.reset()
maze228_env.render()

left_from_reward = maze228_env.env.maze.perception()

print(f'Perception: {"".join(left_from_reward)}')


■ ■ ■ ■ ■ ■ ■
■ □ ■ □ ■ □ ■
■ □ □ □ □ □ ■
■ □ ■ ■ □ □ ■
■ □ ■ □ □ □ ■
■ □ □ □ A $ ■
■ ■ ■ ■ ■ ■ ■
Perception: 009111000


In [21]:
print(f'Total MACS population size: {len(macs_agent.population)}')
s = sorted(macs_agent.population.form_match_set(left_from_reward), key=lambda cl: cl.action)

action_mapping = ['STEP_AHEAD', 'ROTATE_LEFT', 'ROTATE_RIGHT']

for action, group_cl in groupby(s, key=lambda cl: cl.action):
    print(f'\nAction: {action_mapping[action]}, anticipation: {list((macs_agent.get_anticipations(left_from_reward, action)))}')
    for cl in sorted(group_cl, key=lambda cl: cl.effect):
        print(f'\t{cl.condition} {cl.action} {cl.effect}')

Total MACS population size: 211

Action: STEP_AHEAD, anticipation: [0 0 0 9 0 0 0 1 0]
	0#9###### 0 0????????
	#0###10## 0 ?0???????
	0#9##1### 0 ?0???????
	#0####### 0 ??0??????
	0#9###### 0 ???9?????
	0######## 0 ????0????
	0#####0## 0 ?????0???
	#######0# 0 ??????0??
	##9###### 0 ??????0??
	##91#10## 0 ???????1?
	0######## 0 ????????0

Action: ROTATE_LEFT, anticipation: [0 0 0 0 9 1 1 1 0]
	######0## 1 0????????
	#######0# 1 ?0???????
	0######## 1 ??0??????
	#0####### 1 ???0?????
	##9###### 1 ????9????
	###1##### 1 ?????1???
	####1#### 1 ??????1??
	#####1### 1 ???????1?
	######### 1 ????????0

Action: ROTATE_RIGHT, anticipation: [9 1 1 1 0 0 0 0 0]
	##9###### 2 9????????
	###1##### 2 ?1???????
	####1#### 2 ??1??????
	#####1### 2 ???1?????
	######0## 2 ????0????
	#######0# 2 ?????0???
	0######## 2 ??????0??
	#0####### 2 ???????0?
	######### 2 ????????0


Comparing the ACS2 agent in the same environment

In [22]:
def _acs2_metrics(agent: acs2.ACS2, env):
    population = agent.population
    reliable = [cl for cl in population if cl.is_reliable()]
    return {
        'pop': len(population),
        'rel': len(reliable)
    }

@get_from_cache_or_run(cache_path=f'{cache_dir}/acs2_maze228.dill')
def run_acs2_in_maze228(env):
    cfg = acs2.Configuration(classifier_length=9,
                        number_of_possible_actions=3,
                        metrics_trial_frequency=1,
                        user_metrics_collector_fcn=_acs2_metrics,
                        do_ga=False)

    agent = acs2.ACS2(cfg)
    metrics = agent.explore(env, 5000)

    return agent, metrics

acs2_agent, acs2_metrics = run_acs2_in_maze228(maze228_env)

In [23]:
reliable_classifiers = [cl for cl in acs2_agent.population if cl.is_reliable()]

print(f'Total reliable population for ACS2: {len(reliable_classifiers)}')
for cl in acs2_agent.population.form_match_set(left_from_reward):
    if cl.is_reliable():
        print(f'{cl.condition} {cl.action} {cl.effect}')

Total reliable population for ACS2: 402
##911#00# 1 ##009#11#
##9111#0# 0 ##0900#1#
009#11### 2 911#00###


---

**Software packages used**

In [24]:
import session_info
session_info.show()